In [2]:
import findspark

In [3]:
findspark.init('/home/dhruva_stark/spark-3.4.1-bin-hadoop3')

In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark=SparkSession.builder.appName('nlp').getOrCreate()

23/07/12 18:52:55 WARN Utils: Your hostname, linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/12 18:52:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/12 18:52:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/12 18:52:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/12 18:52:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/07/12 18:52:58 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [7]:
data=spark.read.csv('smsspamcollection/SMSSpamCollection',inferSchema=True,sep='\t')

In [8]:
data=data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [9]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import length

In [11]:
data=data.withColumn('length',length(data['text']))

In [12]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [13]:
data.groupBy('class').mean().show()


+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [14]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer)

In [15]:
tokenizer=Tokenizer(inputCol='text',outputCol='token_text')
stop_remove=StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec=CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf=IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric=StringIndexer(inputCol='class',outputCol='label')

In [16]:
from pyspark.ml.feature import VectorAssembler

In [17]:
clean_up=VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [18]:
from pyspark.ml import Pipeline

In [19]:
data_prep_pipe=Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [20]:
cleaner=data_prep_pipe.fit(data)

In [21]:
clean_data=cleaner.transform(data)

In [22]:
clean_data=clean_data.select('label','features')

In [23]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [26]:
from pyspark.ml.regression import LinearRegression


In [27]:
lr=LinearRegression()

In [28]:
training,test=clean_data.randomSplit([0.7,0.3])

In [29]:
spam_detection=lr.fit(training)

23/07/12 18:57:38 WARN DAGScheduler: Broadcasting large task binary with size 1162.6 KiB
23/07/12 18:57:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/12 18:57:40 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:57:41 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:57:42 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:57:43 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:57:43 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:57:44 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:57:44 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:57:44 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:57:45 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/

23/07/12 18:58:02 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:02 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:02 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:02 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:03 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:03 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:03 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:03 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:03 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:03 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:04 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 18:58:04 WAR

In [36]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [30]:
test_results=spam_detection.transform(test)

In [31]:
test_results.show()

23/07/12 18:58:17 WARN DAGScheduler: Broadcasting large task binary with size 1251.7 KiB


+-----+--------------------+--------------------+
|label|            features|          prediction|
+-----+--------------------+--------------------+
|  0.0|(13424,[0,1,2,7,8...| 0.03133317639596479|
|  0.0|(13424,[0,1,4,50,...|-0.08366514075614026|
|  0.0|(13424,[0,1,7,15,...|-0.02796085969110...|
|  0.0|(13424,[0,1,9,14,...|1.115058360024942...|
|  0.0|(13424,[0,1,14,31...|0.011361030792179663|
|  0.0|(13424,[0,1,16,19...|0.031273976995658065|
|  0.0|(13424,[0,1,21,27...| 0.14471220950340588|
|  0.0|(13424,[0,1,23,63...|-0.03959301744734...|
|  0.0|(13424,[0,1,24,31...|-0.08352035682637764|
|  0.0|(13424,[0,1,31,43...|-0.00379311160504...|
|  0.0|(13424,[0,1,43,69...|-0.00580462307025...|
|  0.0|(13424,[0,1,150,1...| 0.02094489775988382|
|  0.0|(13424,[0,1,428,6...|0.017544311215045738|
|  0.0|(13424,[0,1,500,5...|7.257212641205227E-4|
|  0.0|(13424,[0,1,881,1...|0.001967781978055...|
|  0.0|(13424,[0,2,3,6,9...|0.001589961106813...|
|  0.0|(13424,[0,2,4,5,7...|0.016166573048769835|


In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [33]:
acc_eval=MulticlassClassificationEvaluator()

In [34]:
acc = acc_eval.evaluate(test_results)



23/07/12 18:58:36 WARN DAGScheduler: Broadcasting large task binary with size 1256.6 KiB


In [38]:
lr_model=lr.fit(training)

23/07/12 19:05:46 WARN DAGScheduler: Broadcasting large task binary with size 1162.6 KiB
23/07/12 19:05:47 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:48 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:48 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:48 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:48 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:49 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:49 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:49 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:49 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:50 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:05:50 WAR

23/07/12 19:06:06 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:06 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:06 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:06 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:06 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:07 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:07 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:07 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:07 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:07 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:08 WARN DAGScheduler: Broadcasting large task binary with size 1163.2 KiB
23/07/12 19:06:08 WAR

In [39]:
final_results=lr_model.evaluate(test)

23/07/12 19:06:20 WARN DAGScheduler: Broadcasting large task binary with size 1257.2 KiB


In [40]:
final_results.residuals.show()

23/07/12 19:06:44 WARN DAGScheduler: Broadcasting large task binary with size 1254.7 KiB


+--------------------+
|           residuals|
+--------------------+
|-0.03133317639596479|
| 0.08366514075614026|
|0.027960859691103717|
|-1.11505836002494...|
|-0.01136103079217...|
|-0.03127397699565...|
|-0.14471220950340588|
|0.039593017447342026|
| 0.08352035682637764|
|0.003793111605047207|
|0.005804623070251057|
|-0.02094489775988382|
|-0.01754431121504...|
|-7.25721264120522...|
|-0.00196778197805...|
|-0.00158996110681...|
|-0.01616657304876...|
|-9.38754233634764...|
|-0.05892532182827735|
|-0.12973929482398422|
+--------------------+
only showing top 20 rows



In [41]:
final_results.rootMeanSquaredError

0.1516867503194918

In [42]:
final_results.r2


0.7952903272920879